<a href="https://colab.research.google.com/github/luisitobarcito/DiffProgTF/blob/master/diff_programming_boolean_func.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%matplotlib inline 

import tensorflow as tf 
import numpy as np
import matplotlib.pyplot as plt

# Learning  Boolean functions by example.
Even though this might look like an overkill, it will be helpful to explain the concept of learning by example and the modular perspective. Let's say we want to learn two basic functions: the AND, and the OR functions.

## Define a generic tuneable function.
Our generic parameterized function module will model binary input to binary output vectors. Rather than producing binary digits, the generic function can output a real number between o and 1. These outputs can be interpreted as the probability of the output being 1, which also correspond to the expected value of the output under the Bernoulli distribution. Our function corresponds to an affine function followed by a squashing nonlinearlity such as logistic sigmoid.
\begin{equation}
f(x) = \sigma(Wx + b)
\end{equation}
where $\sigma(z) = 1/(1+e^{-z})$.

In [0]:
class GenFunc(object):
    def __init__(self, n_in, n_out=1, act_func=None):
        self.W = tf.Variable(tf.random_normal([n_in, n_out], stddev=0.35), dtype=tf.float32)
        self.b = tf.Variable(tf.zeros([n_out], dtype=tf.float32), dtype=tf.float32)
        if act_func is None:
            self.act_func = tf.sigmoid
        else:
            self.act_func = act_func

    def __call__(self, X, is_logit=False):
        # For numerical stability, We can select logit output 
        # to train the function with cross-entropy 
        if is_logit is False:
            return self.act_func(tf.add(tf.matmul(X, self.W), self.b))
        else:
            return tf.add(tf.matmul(X, self.W), self.b)


## Define a differentiable objective
We use cross entropy function to evaluate the match of the output produced by our tuneable function and the desired output of our program. In the context of binary digits and our tuneable function representing the probability of being 1, the cross entropy function is also a likelihood function of the parameters given the desired input-output data.


In [0]:
def learnProgram(sess, f, train_in, train_out, n_iter=10000):
    X = tf.placeholder(shape=train_in.shape, dtype=tf.float32)
    Y = tf.placeholder(shape=train_out.shape, dtype=tf.float32)      
    ## The cross entropy loss applied to the function to be tuned
    loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=Y, logits=f(X, is_logit=True)))
    ## Training procedure
    trainer = tf.train.GradientDescentOptimizer(learning_rate=0.2)
    train_step = trainer.minimize(loss)
    tf.global_variables_initializer().run()
    for iTr in range(n_iter):
        sess.run(train_step, feed_dict={X: train_in, Y: train_out})

## The OR function
The data for the OR function is given by the following truth table:

| X1 | X2 | X1 OR X2 |
|----|:--:|:--------:|
| 0  | 0  |    0     |
| 0  | 1  |    1     |
| 1  | 0  |    1     |
| 1  | 1  |    1     |


In [4]:
## Create our set of input output pairs
X = np.asarray([[0, 0],[0, 1],[1, 0],[1, 1]], dtype=np.float32)
Y_or = np.asarray([0, 1, 1, 1], dtype=np.float32)[:,None]
## Let's display our set of input output pairs
from tabulate import tabulate
print(tabulate (np.concatenate((X,Y_or), axis=1), headers=['X1','X2', 'X1 OR X2']))

  X1    X2    X1 OR X2
----  ----  ----------
   0     0           0
   0     1           1
   1     0           1
   1     1           1


In [5]:
## Instantiate a tuneable function 
F = GenFunc(2, 1)
## call the tuning routine  within a TF session
with tf.Session() as sess:
    learnProgram(sess, F, X, Y_or)
    ## display the outputs of the function after tuning
    Z = F(X).eval()
    print(tabulate(np.concatenate((X,Z), axis=1), headers=['X1','X2', 'F(X)']))

Instructions for updating:
Colocations handled automatically by placer.
  X1    X2       F(X)
----  ----  ---------
   0     0  0.0101954
   0     1  0.995924
   1     0  0.995925
   1     1  1


## The AND function
The data for the AND function is given by the following truth table:

| X1 | X2 | X1 AND X2 |
|----|:--:|:---------:|
| 0  | 0  |    0      |
| 0  | 1  |    0      |
| 1  | 0  |    0      |
| 1  | 1  |    1      |

In [6]:
Y_and = np.asarray([0, 0, 0, 1], dtype=np.float32)[:,None]
F = GenFunc(2, 1)
with tf.Session() as sess:
    learnProgram(sess, F, X, Y_and)
    Z = F(X).eval()
    print (tabulate(np.concatenate((X,Z), axis=1), headers=['X1','X2', 'F(X)']))

  X1    X2         F(X)
----  ----  -----------
   0     0  1.51419e-06
   0     1  0.0101462
   1     0  0.0101462
   1     1  0.985793


## The XOR function
The data for the XOR function is given by the following truth table:

| X1 | X2 | X1 XOR X2 |
|----|:--:|:---------:|
| 0  | 0  |    0      |
| 0  | 1  |    1      |
| 1  | 0  |    1      |
| 1  | 1  |    0      |


In [7]:
Y_xor = np.asarray([0, 1, 1, 0], dtype=np.float32)[:,None]
F = GenFunc(2, 1)
with tf.Session() as sess:
    learnProgram(sess, F, X, Y_xor)
    Z = F(X).eval()
    print (tabulate(np.concatenate((X,Z), axis=1), headers=['X1','X2', 'F(X)']))

  X1    X2    F(X)
----  ----  ------
   0     0     0.5
   0     1     0.5
   1     0     0.5
   1     1     0.5


### What happened?
Note that for the XOR the function tuning did not succed in replicating the input output relation.
It turns out that our tunable function can only represent a subset of all binary functions. 
### Composing tunable modules to increase the expresivity
Fortunately, our tunable function does represent the basic Boolean functions. Compositions of basic Boolean functions can give us all Boolean functions from 2 inputs to 1 output.  

In [8]:
F1 = GenFunc(2, 1)
F2 = GenFunc(2, 1)
F3 = GenFunc(2, 1)
## Define a composition of tunable functions
def G(X, is_logit=False):
    Z1 = F1(X)
    Z2 = F2(X)
    Z3 = F3(tf.concat((Z1, Z2), axis=1), is_logit=is_logit)
    return Z3

with tf.Session() as sess:
    learnProgram(sess, G, X, Y_xor)
    ## Show the tuned composition of functions G(X)
    Z = G(X).eval()
    print("\n Tuned composition")
    print (tabulate(np.concatenate((X,Z), axis=1), headers=['X1','X2', 'G(X)']))
    ## Show the tuned module F1(X)
    Z1 = F1(X).eval()
    print("\n Tuned input module F1")
    print( tabulate(np.concatenate((X,Z1), axis=1), headers=['X1','X2', 'F1(X)']))
    ## Show the tuned module F2(X)
    Z2 = F2(X).eval()
    print("\n Tuned input module F2")
    print( tabulate(np.concatenate((X,Z2), axis=1), headers=['X1','X2', 'F2(X)']))
    ## Show the intermediate tuned module F3(X)
    print("\n Tuned intermediate module F3")
    Z3 = F3(X).eval()
    print( tabulate(np.concatenate((X,Z3), axis=1), headers=['Z1','Z2', 'F3(Z)']))
   
sess.close()


 Tuned composition
  X1    X2       G(X)
----  ----  ---------
   0     0  0.0284485
   0     1  0.976936
   1     0  0.97693
   1     1  0.026026

 Tuned input module F1
  X1    X2       F1(X)
----  ----  ----------
   0     0  0.00140859
   0     1  0.0933116
   1     0  0.0932741
   1     1  0.882426

 Tuned input module F2
  X1    X2      F2(X)
----  ----  ---------
   0     0  0.0611437
   0     1  0.972814
   1     0  0.972744
   1     1  0.999949

 Tuned intermediate module F3
  Z1    Z2       F3(Z)
----  ----  ----------
   0     0  0.0168781
   0     1  0.992532
   1     0  1.111e-06
   1     1  0.00852749
